In [1]:
import torch
from torch import nn
from torchvision import datasets
import numpy as np
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
import torchvision.transforms

In [2]:
training_data = datasets.MNIST(root="data",
                              train=True,
                              download=True,
                              transform=torchvision.transforms.ToTensor())

C:\Users\Pete\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
test_data = datasets.MNIST(root="data",
                          train=False,
                          download=True,
                          transform=torchvision.transforms.ToTensor())

In [4]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [5]:
for X, y in test_dataloader:
    print(X.shape)
    print(y.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=12, kernel_size=2, stride=1, padding=1),
                                  nn.ReLU())
        self.conv2 = nn.Sequential(nn.Conv2d(in_channels=12, out_channels=24, kernel_size=2, stride=1, padding=1),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size=2))
        self.conv3 = nn.Sequential(nn.Conv2d(in_channels=24, out_channels=48, kernel_size=2, stride=1, padding=1),
                                  nn.ReLU())
        self.conv4 = nn.Sequential(nn.Conv2d(in_channels=48, out_channels=96, kernel_size=2, stride=1, padding=1),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size=2))
        self.flatten = nn.Flatten()
        self.linear = nn.Sequential(nn.Linear(in_features=(6144), out_features=512),
                                   nn.ReLU(),
                                   nn.Linear(in_features=(512), out_features=512),
                                   nn.ReLU(),
                                   nn.Linear(in_features=512,out_features=10))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.flatten(x)
        x = self.linear(x)
        return x
        
model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (conv1): Sequential(
    (0): Conv2d(1, 12, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(12, 24, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(24, 48, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv4): Sequential(
    (0): Conv2d(48, 96, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Sequential(
    (0): Linear(in_features=6144, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)

In [17]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss {loss:>7f} [{current:>5d}/{size:>5d}]")

In [18]:
def test(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    model.eval()
    
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [19]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1} \n---------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1 
---------------------
loss 2.305007 [    0/60000]
loss 0.172099 [ 6400/60000]
loss 0.158488 [12800/60000]
loss 0.116808 [19200/60000]
loss 0.058603 [25600/60000]
loss 0.161858 [32000/60000]
loss 0.085256 [38400/60000]
loss 0.106100 [44800/60000]
loss 0.156642 [51200/60000]
loss 0.077662 [57600/60000]
Test Error: 
 Accuracy: 97.4%, Avg loss: 0.095575 

Epoch 2 
---------------------
loss 0.046386 [    0/60000]
loss 0.078777 [ 6400/60000]
loss 0.055260 [12800/60000]
loss 0.067968 [19200/60000]
loss 0.013508 [25600/60000]
loss 0.022018 [32000/60000]
loss 0.053648 [38400/60000]
loss 0.058335 [44800/60000]
loss 0.162152 [51200/60000]
loss 0.021814 [57600/60000]
Test Error: 
 Accuracy: 98.7%, Avg loss: 0.042862 

Epoch 3 
---------------------
loss 0.013539 [    0/60000]
loss 0.041576 [ 6400/60000]
loss 0.035188 [12800/60000]
loss 0.083531 [19200/60000]
loss 0.015271 [25600/60000]
loss 0.003226 [32000/60000]
loss 0.009197 [38400/60000]
loss 0.027943 [44800/60000]
loss 0.251126 [5120